This notebook parses sgf files generated by the `match` command,
and loads them into a Pandas dataframe.

### Load libraries

In [2]:
import dataclasses
import os
import random
import re
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map

from go_attack import game_info

In [3]:
# this should be "/nas/ucb/ttseng/adv-checkpoints/37-to-61/sgfs" but I'm testing this on my local machine
MATCH_DIR = "/Users/t/code/far/adv-checkpoints/37-to-61/sgfs"

sgf_paths = game_info.find_sgf_files(MATCH_DIR)
raw_sgf_strs = game_info.read_and_concat_all_files(sgf_paths)

len(raw_sgf_strs)

  0%|          | 0/1400 [00:00<?, ?it/s]

2500

In [4]:
random.seed(42)
game_infos: List[game_info.GameInfo] = process_map(
    game_info.parse_game_info,
    raw_sgf_strs,
    max_workers=64,
    chunksize=50,
)

  0%|          | 0/2500 [00:00<?, ?it/s]

In [5]:
%%time
df = pd.DataFrame([gi.to_dict() for gi in game_infos])
print("gtypes:", df.gtype.unique())
print("Number of cleanup games:", (df.gtype == "cleanuptraining").sum())

# Filter to only normal games
df = df[(df.gtype == "normal")]
print("Fraction continuation:    ", df.is_continuation.mean())
print("Fraction used_initial_pos:", df.used_initial_position.mean())
print("max(init_turn_num)       :", df.init_turn_num.max())

df.head()

gtypes: ['normal']
Number of cleanup games: 0
Fraction continuation:     0.0
Fraction used_initial_pos: 0.0
max(init_turn_num)       : 0
CPU times: user 131 ms, sys: 7.75 ms, total: 139 ms
Wall time: 154 ms


,board_size,gtype,start_turn_idx,init_turn_num,used_initial_position,b_name,w_name,win_color,komi,handicap,...,score_rule,tax_rule,sui_legal,has_button,whb,fpok,sgf_str,lose_color,win_name,lose_name
0,19,normal,0,0,False,bot-cp505-v2,adv-s56046848-d13775568-v600,w,6.5,0,...,AREA,NONE,True,False,0,False,(;FF[4]GM[1]SZ[19]PB[bot-cp505-v2]PW[adv-s5604...,b,adv-s56046848-d13775568-v600,bot-cp505-v2
1,19,normal,0,0,False,adv-s46847744-d11540675-v600,bot-cp127-v1,w,6.5,0,...,AREA,NONE,True,False,0,False,(;FF[4]GM[1]SZ[19]PB[adv-s46847744-d11540675-v...,b,bot-cp127-v1,adv-s46847744-d11540675-v600
2,19,normal,0,0,False,bot-cp127-v1,adv-s55047168-d13516907-v600,b,6.5,0,...,AREA,NONE,True,False,0,False,(;FF[4]GM[1]SZ[19]PB[bot-cp127-v1]PW[adv-s5504...,w,bot-cp127-v1,adv-s55047168-d13516907-v600
3,19,normal,0,0,False,bot-cp127-v1,adv-s46847744-d11540675-v600,b,6.5,0,...,AREA,NONE,True,False,0,False,(;FF[4]GM[1]SZ[19]PB[bot-cp127-v1]PW[adv-s4684...,w,bot-cp127-v1,adv-s46847744-d11540675-v600
4,19,normal,0,0,False,bot-cp127-v1,adv-s40074240-d9782343-v600,w,6.5,0,...,AREA,NONE,True,False,0,False,(;FF[4]GM[1]SZ[19]PB[bot-cp127-v1]PW[adv-s4007...,b,adv-s40074240-d9782343-v600,bot-cp127-v1


In [6]:
pattern = re.compile("C\\[([^]]+?)];([BW])\\[]C\\[([^]]+?)]\\)$")
result = pattern.search(df.iloc[0].sgf_str)
print(df.iloc[0].sgf_str[-80:])
print(result.group(1), "   ", result.group(2), "    ", result.group(3))

];B[]C[0.00 1.00 0.00 -117.3 v=1];W[]C[1.00 0.00 0.00 93.1 v=600 result=W+92.5])
0.00 1.00 0.00 -117.3 v=1     W      1.00 0.00 0.00 93.1 v=600 result=W+92.5


In [7]:
advs = [
 "adv-s61751296-d15255445-v600", 
 "adv-s60753920-d15021246-v600",  
 "adv-s59897344-d14761403-v600",  
 "adv-s59042304-d14514443-v600",  
 "adv-s58043904-d14307163-v600",  
]
adv_mask = pd.Series(False, index=df.index)
for adv in advs:
    adv_mask = adv_mask | (df.b_name == adv) | (df.w_name == adv)
victim = "bot-cp505-v2"
victim_mask = (df.b_name == victim) | (df.w_name == victim)
filtered_df = df.loc[adv_mask & victim_mask]

In [43]:
def get_predictions(sgf: str, adv_color: str):    
    # returns:
    # - victim's prediction for own win rate
    # - victim's prediction for own score
    # - adv's prediction for own win rate
    # - adv's prediction for own score
    # - actual adv score
    #
    # `sgf` should be a game that ends via the victim passing and then the adversary passing
    matches = pattern.search(sgf)
    last_move_color = matches.group(2).lower()
    if last_move_color != adv_color:
        raise Exception("WARNING: last_move_color != adv_color")
    victim_comment = matches.group(1).split()
    adv_comment = matches.group(3).split()
    score_str = adv_comment[-1].replace("result=", "")
    white_score = float(score_str[2:]) if score_str[0] == "W" else -float(score_str[2:])
    if adv_color == "b":
        victim_win_rate = float(victim_comment[0])
        victim_value = float(victim_comment[3])
        adv_win_rate = float(adv_comment[1])
        adv_value = -float(adv_comment[3])
        adv_score = -white_score
    elif adv_color == "w":
        victim_win_rate = float(victim_comment[1])
        victim_value = -float(victim_comment[3])
        adv_win_rate = float(adv_comment[0])
        adv_value = float(adv_comment[3])
        adv_score = white_score
    else:
        raise ValueException(f"unexpected adv_color: {adv_color}")
    return victim_win_rate, victim_value, adv_win_rate, adv_value, adv_score 
    
victim_win_predictions = []
victim_value_predictions = []
adv_win_predictions = []
adv_value_predictions = []
actual_adv_scores = []
for index, game in filtered_df.iterrows():
    adv_color = "b" if game.b_name.startswith("adv") else "w"
    if adv_color != game.win_color:
        # print(game.sgf_str, "\n\n")
        continue
    victim_win_rate, victim_value, adv_win_rate, adv_value, adv_score = get_predictions(game.sgf_str, adv_color)
    victim_win_predictions.append(victim_win_rate)
    victim_value_predictions.append(victim_value)
    adv_win_predictions.append(adv_win_rate)
    adv_value_predictions.append(adv_value)
    actual_adv_scores.append(adv_score)
    
# add in stats (which I just manually extracted) for the games where the adversary lost.
# In these games, the game ends by playing to the end, not via passing.
# victim_win_predictions.extend([1., 1., 1.])
# victim_value_predictions.extend([142.8, 260.1, 146.8])
# adv_win_predictions.extend([0., 0., 0.])
# adv_value_predictions.extend([-140.9, -257.7, -143.9])
# actual_adv_scores.extend([-143.5, -257.5, -145.5])

In [44]:
victim_value_mse = np.array([(pred + score) ** 2 for pred, score in zip(victim_value_predictions, actual_adv_scores)]).mean()
adv_value_mse = np.array([(pred - score) ** 2 for pred, score in zip(adv_value_predictions, actual_adv_scores)]).mean()
print(f"MSE: adv: {adv_value_mse:.5f}; victim: {victim_value_mse:.5f}")

MSE: adv: 1.20198; victim: 50346.86429


In [45]:
print("num games:", len(adv_win_predictions))
arrs = [
    ("adv win rate predictions", adv_win_predictions), 
    ("adv value predictions", adv_value_predictions),
    ("victim win rate predictions", victim_win_predictions), 
    ("victim value predictions", victim_value_predictions),
    ("actual adv scores", actual_adv_scores),
]
for title, arr in arrs:
    arr = np.array(arr)
    print(f"{title}:  min={arr.min():.5f}, mean={arr.mean():.5f}, stddev={arr.std():.5f})")

num games: 247
adv win rate predictions:  min=0.97000, mean=0.99931, stddev=0.00359)
adv value predictions:  min=40.00000, mean=90.25020, stddev=11.56980)
victim win rate predictions:  min=1.00000, mean=1.00000, stddev=0.00000)
victim value predictions:  min=72.10000, mean=133.86194, stddev=26.90908)
actual adv scores:  min=37.50000, mean=89.52024, stddev=11.66241)


### Helper functions

In [ ]:
@dataclasses.dataclass
class SGFUrl:
    sgf: str
    text: str

    def sgf_str_to_url(self, sgf_str: str) -> str:
        return f"https://humancompatibleai.github.io/sgf-viewer/#sgf={sgf_str}"

    def __post_init__(self):
        self.url = self.sgf_str_to_url(self.sgf)

    def _repr_html_(self):
        """HTML link to this URL."""
        return f'<a href="{self.url}">{self.text}</a>'

    def __str__(self):
        """Return the underlying string."""
        return self.url

### Analyze data

In [ ]:
plt.subplot(1, 2, 1)
df[(df.b_name == "cp127-v1") & (df.w_name == "cp63-v1024")].win_color.hist()

plt.subplot(1, 2, 2)
df[(df.w_name == "cp127-v1") & (df.b_name == "cp63-v1024")].win_color.hist()

In [ ]:
cur_df = df[(df.win_name == "cp63-v1024") & (df.lose_name == "cp127-v1")]
len(cur_df)

In [ ]:
SGFUrl(sgf=cur_df.sgf_str.iloc[0], text="cp63-v1024 beats cp127-v1 (game2)")